Write PCWAG segments to files, convert to hg38, save back to db, then run Mecan

In [1]:
from pymongo import MongoClient
import os

In [ ]:
db = MongoClient()['pcawg']['samples']

In [ ]:
out_path = '/Volumes/originalData/ICGC/PCAWG_segments'
for sample in db.find():
    out_file = os.path.join(out_path, sample['sample_id'])
    with open(out_file, 'w') as fo:
        print('id\tchro\tstart\tend\tvalue\tprobes', file=fo)
        for seg in sample['segments']:
            print('{}\t{}\t{}\t{}\t{}\t{}'.format(sample['sample_id'], seg['chro'],seg['start'],
                                                  seg['end'], seg['value'], seg['probes']), file=fo)
    

In [ ]:
data = []
for sample in db.find({},{'_id':0}):
    data.append(sample)

In [ ]:
out_path = '/Volumes/originalData/ICGC/PCAWG_segments_hg38'
for d in data:
    file_path = os.path.join(out_path, d['sample_id'])
    # segments
    cnvs = []
    with open(file_path, 'r') as fin:
        next(fin)
        for line in fin:
            line = line.strip().split('\t')
            cnvs.append({'chro':line[1], 'start':int(line[2]), 'end':int(line[3]),
                         'probes':int(line[5]), 'value':float(line[4])})
    d['segments'] = cnvs

In [ ]:
db = MongoClient()['pcawg']['samples_hg38']
db.drop()
db.insert_many(data)

In [2]:
import sys
sys.path.append('/Users/bogao/Desktop/projects/Relative copy number/Python/mecan')
import mecan4cna.algorithms as alg

In [3]:
pcawg_data = []
db = MongoClient()['pcawg']['samples_hg38']
out_path = '/Volumes/originalData/ICGC/PCAWG_segments_hg38'

for sample in db.find():
    if 'Normal' not in sample['sample_type']:
        pcawg_data.append({'source': 'PCAWG',
                          'project': sample['project_code'],
                          'sample_id': sample['sample_id'],
                          'morphology': sample['donor_diagnosis_icd10'],
                          'topography': None,
                          'stage': sample['donor_tumour_stage_at_diagnosis'],
                          'gender': sample['donor_sex'],
                          'age': sample['donor_age_at_diagnosis'],
                          'vital_status': sample['donor_vital_status'],
                          'file_path': os.path.join(out_path, sample['sample_id']),
                          'segments': sample['segments'],
                          'platform': 'WGS'})

In [4]:
m = alg.mecan()
m.peak_thresh=5000
for sample in pcawg_data:
    res = m.run(sample['segments'])
    if len(res) > 1:
        base = res[0]
        level_dist = res[1]
    else:
        base = None
        level_dist = None
    sample['base'] = base
    sample['level_distance'] = level_dist

In [5]:
db = MongoClient()['Rebased']['pcawg']
db.drop()
db.insert_many(pcawg_data)